# VidGen 🎬 - AI-Powered Video Generation in Google Colab

This notebook demonstrates how to use VidGen to create AI-generated videos from text scripts using Google Colab's free GPU resources.

## 🚀 Features
- **AI Script Processing**: Powered by Google Gemini
- **Image Generation**: Stable Diffusion for character images
- **Voice Synthesis**: Bark TTS for natural speech
- **Audio Generation**: Procedural background music and effects
- **Video Assembly**: Professional transitions and effects

## 📋 Prerequisites
1. **GPU Runtime**: Enable GPU in Runtime → Change runtime type
2. **API Key**: Get a Google Gemini API key from [Google AI Studio](https://makersuite.google.com/)

---

## 1. 📦 Setup and Installation

Run this cell to install VidGen and all dependencies in Colab.

In [ ]:
# Check GPU availability
!nvidia-smi 2>/dev/null || echo "⚠️ GPU not available, using CPU mode"

# Method 1: Try cloning from GitHub (if repository is public)
import os
import subprocess
import sys

try:
    print("🔄 Attempting to clone VidGen repository...")
    result = subprocess.run(['git', 'clone', 'https://github.com/AhmedWGabr/VidGen.git'], 
                          capture_output=True, text=True, timeout=30)
    
    if result.returncode == 0:
        print("✅ Successfully cloned from GitHub")
        %cd VidGen
        
        # Add to Python path immediately
        if '/content/VidGen/src' not in sys.path:
            sys.path.insert(0, '/content/VidGen/src')
        
        !python setup_colab.py
        
        # Verify the installation worked
        try:
            import vidgen
            print("✅ VidGen module imported successfully")
        except ImportError:
            print("⚠️ Module import failed, setting up manually...")
            raise Exception("Import failed")
    else:
        print("❌ GitHub clone failed, using manual setup...")
        raise Exception("Clone failed")
        
except Exception as e:
    print("🔧 Setting up VidGen manually...")
    
    # Create project structure
    !mkdir -p VidGen/src/vidgen/{core,models,services,ui,utils}
    %cd VidGen
    
    # Add to Python path
    sys.path.insert(0, '/content/VidGen/src')
    
    # Download setup files directly
    print("📦 Installing dependencies...")
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install diffusers transformers accelerate bark scipy pillow opencv-python-headless moviepy
    !pip install google-generativeai xformers optimum psutil
    
    # Install system dependencies
    !apt-get update -qq
    !apt-get install -y ffmpeg libsndfile1
    
    print("✅ Manual setup complete!")

# Final verification
print("\n🔍 Verifying Python environment...")
print(f"📍 Current directory: {os.getcwd()}")
print(f"🐍 Python path includes: {[p for p in sys.path if 'VidGen' in p]}")

# Test imports
try:
    import torch
    print(f"✅ PyTorch: {torch.__version__} (CUDA: {torch.cuda.is_available()})")
except ImportError:
    print("❌ PyTorch not available")

try:
    import transformers
    print(f"✅ Transformers: {transformers.__version__}")
except ImportError:
    print("❌ Transformers not available")

print("\n🎯 Setup verification complete!")

## 2.5. 🔧 VidGen Core Code Setup

If the repository clone didn't work, run this cell to set up the VidGen core functionality:

In [ ]:
# Enhanced VidGen package structure and core files setup
import os
import sys

# Ensure we're in the right directory and path is set
current_dir = os.getcwd()
print(f"📍 Working directory: {current_dir}")

# Add to Python path if not already there
src_path = os.path.join(current_dir, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"🐍 Added to Python path: {src_path}")

# Create __init__.py files
init_files = [
    'src/__init__.py',
    'src/vidgen/__init__.py', 
    'src/vidgen/core/__init__.py',
    'src/vidgen/models/__init__.py',
    'src/vidgen/services/__init__.py',
    'src/vidgen/ui/__init__.py',
    'src/vidgen/utils/__init__.py'
]

for init_file in init_files:
    os.makedirs(os.path.dirname(init_file), exist_ok=True)
    with open(init_file, 'w') as f:
        if 'vidgen/__init__.py' == init_file.split('/')[-2:]:
            # Main package init
            f.write('"""VidGen - AI-Powered Video Generation Framework"""\n__version__ = "0.1.0"\n')
        else:
            f.write('')

# Create a minimal VidGen configuration
config_code = '''
"""VidGen Configuration Module"""
import os
from typing import Dict, Any, Optional
from dataclasses import dataclass, field

@dataclass
class VideoGenConfig:
    """Configuration class for VidGen video generation."""
    
    # API Configuration
    gemini_api_key: str = field(default_factory=lambda: os.getenv('GEMINI_API_KEY', ''))
    
    # Model Configuration
    image_model: str = 'stabilityai/stable-diffusion-2-1'
    voice_model: str = 'suno/bark'
    
    # Output Configuration
    output_dir: str = 'colab_output'
    temp_dir: str = 'temp'
    
    # Video Configuration
    fps: int = 24
    resolution: tuple = (1280, 720)
    duration_per_scene: float = 3.0
    
    # Processing Configuration
    batch_size: int = 1
    device: str = 'auto'
    
    def __post_init__(self):
        """Initialize configuration after object creation."""
        if self.device == 'auto':
            try:
                import torch
                self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            except ImportError:
                self.device = 'cpu'
        
        # Create directories
        os.makedirs(self.output_dir, exist_ok=True)
        os.makedirs(self.temp_dir, exist_ok=True)
'''

# Write configuration file
with open('src/vidgen/core/config.py', 'w') as f:
    f.write(config_code)

# Create a comprehensive main module for Colab demo
main_code = '''
"""VidGen Main Module - Colab Demo Version"""
import os
import sys
from typing import Dict, Any, Optional

# Handle relative imports for Colab
try:
    from .core.config import VideoGenConfig
except ImportError:
    sys.path.append(os.path.dirname(os.path.dirname(__file__)))
    from vidgen.core.config import VideoGenConfig

def main(script: str, config: Optional[VideoGenConfig] = None) -> str:
    """
    Generate a video from a text script - Colab Demo Version.
    
    Args:
        script: The input script text
        config: Configuration object
        
    Returns:
        Path to the generated video file
    """
    if config is None:
        config = VideoGenConfig()
    
    print("🎬 VidGen Demo - Processing script...")
    print(f"📝 Script preview: {script[:100]}...")
    print(f"🎮 Using device: {config.device}")
    
    # For Colab demo, we'll create a simple demonstration
    output_path = os.path.join(config.output_dir, 'demo_video.mp4')
    
    # Create a simple demo video using OpenCV
    try:
        import cv2
        import numpy as np
        
        print("🎥 Creating demo video with OpenCV...")
        
        # Create a simple demo video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, config.fps, config.resolution)
        
        # Create frames for a 5-second demo
        total_frames = int(config.fps * 5)
        
        for frame_num in range(total_frames):
            # Create a gradient background
            frame = np.zeros((config.resolution[1], config.resolution[0], 3), dtype=np.uint8)
            
            # Create a moving gradient effect
            for y in range(config.resolution[1]):
                color_intensity = int(50 + 30 * np.sin(y / 50 + frame_num / 10))
                frame[y, :] = [color_intensity, color_intensity//2, min(255, color_intensity*2)]
            
            # Add main title
            title = "VidGen AI Demo"
            cv2.putText(frame, title, (config.resolution[0]//2 - 200, 150), 
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
            
            # Add progress indicator
            progress = f"Frame {frame_num + 1}/{total_frames}"
            cv2.putText(frame, progress, (50, config.resolution[1] - 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 200, 200), 2)
            
            # Add script preview
            script_preview = script.split('\n')[0][:50] if script else "Sample Script"
            cv2.putText(frame, f"Script: {script_preview}", (50, config.resolution[1]//2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (150, 255, 150), 2)
            
            out.write(frame)
        
        out.release()
        print(f"✅ Demo video created: {output_path}")
        return output_path
        
    except Exception as e:
        print(f"❌ Error creating demo video: {e}")
        print("📝 Script processed successfully (video generation skipped in demo)")
        return "demo_completed"

def generate_video_from_script(script_text: str, **kwargs) -> str:
    """Alternative entry point for video generation."""
    config = VideoGenConfig()
    for key, value in kwargs.items():
        if hasattr(config, key):
            setattr(config, key, value)
    return main(script_text, config)

# For direct testing
if __name__ == "__main__":
    test_script = "VidGen Test: AI-powered video generation demo"
    result = main(test_script)
    print(f"Test completed: {result}")
'''

# Write main module
with open('src/vidgen/main.py', 'w') as f:
    f.write(main_code)

# Create a simple services module for Gemini API (placeholder)
gemini_code = '''
"""Gemini API Service - Placeholder for Colab Demo"""

def call_gemini_api(script: str, api_key: str, **kwargs) -> str:
    """Placeholder Gemini API call for demo purposes."""
    print("🤖 Simulating Gemini API call...")
    print(f"API Key configured: {'Yes' if api_key else 'No'}")
    
    # Return a simple structured response for demo
    return f"Processed script: {script[:100]}..."
'''

with open('src/vidgen/services/gemini_api.py', 'w') as f:
    f.write(gemini_code)

# Test the module creation
try:
    # Reload the module if it was already imported
    if 'vidgen' in sys.modules:
        import importlib
        importlib.invalidate_caches()
    
    import vidgen
    from vidgen.core.config import VideoGenConfig
    from vidgen.main import main as vidgen_main
    
    print("✅ VidGen modules created and imported successfully!")
    print(f"📚 Package location: {vidgen.__file__ if hasattr(vidgen, '__file__') else 'In-memory'}")
    
    # Quick test
    test_config = VideoGenConfig()
    print(f"📋 Test config created - Device: {test_config.device}")
    
except Exception as e:
    print(f"❌ Error importing created modules: {e}")
    print("Modules created but import failed - this is normal in some Colab setups")

print("✅ VidGen core code setup complete!")
print("📁 Created package structure and essential modules")
print("🔄 Ready for video generation demo!")

## 3. 🔑 Configure API Keys

Set up your Google Gemini API key for script processing.

In [ ]:
import os

# Try different methods to get the API key
GEMINI_API_KEY = None

# Option 1: Use Colab Secrets (Recommended)
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except Exception as e:
    print(f"⚠️ Colab secrets not available: {e}")

# Option 2: Check environment variables
if not GEMINI_API_KEY:
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
    if GEMINI_API_KEY:
        print("✅ API key found in environment variables")

# Option 3: Manual input (less secure)
if not GEMINI_API_KEY:
    try:
        from getpass import getpass
        GEMINI_API_KEY = getpass("Enter your Gemini API Key (get it from https://makersuite.google.com/): ")
        if GEMINI_API_KEY:
            print("✅ API key entered manually")
    except Exception as e:
        print(f"❌ Manual input failed: {e}")

# Option 4: Use demo mode without API key
if not GEMINI_API_KEY:
    print("⚠️ No API key provided - running in demo mode")
    print("📝 To get full functionality, get your API key from: https://makersuite.google.com/")
    GEMINI_API_KEY = "demo_mode"

# Set environment variable for other modules
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
print("🔑 API key configured successfully")

# Display setup status
print("\n📊 Setup Status:")
print(f"  • API Key: {'Configured' if GEMINI_API_KEY and GEMINI_API_KEY != 'demo_mode' else 'Demo Mode'}")
print(f"  • Working Directory: {os.getcwd()}")
print(f"  • Python Path: {'VidGen modules accessible' if any('VidGen' in p for p in sys.path) else 'Standard path'}")

try:
    import torch
    print(f"  • GPU Available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"  • GPU Device: {torch.cuda.get_device_name(0)}")
except ImportError:
    print("  • GPU Status: PyTorch not available")

## 4. 🎬 Generate Your First Video

Let's create a simple video from a text script.

In [ ]:
# Enhanced VidGen module import with fallback
import sys
import os

print("🚀 Initializing VidGen for video generation...")

# Try multiple import strategies
vidgen_main = None
VideoGenConfig = None

# Strategy 1: Direct import
try:
    from vidgen.main import main as vidgen_main
    from vidgen.core.config import VideoGenConfig
    print("✅ VidGen modules imported successfully (direct import)")
except ImportError as e:
    print(f"⚠️ Direct import failed: {e}")
    
    # Strategy 2: Add path and retry
    try:
        current_dir = os.getcwd()
        src_path = os.path.join(current_dir, 'src')
        if src_path not in sys.path:
            sys.path.insert(0, src_path)
        
        # Clear import cache
        import importlib
        importlib.invalidate_caches()
        
        from vidgen.main import main as vidgen_main
        from vidgen.core.config import VideoGenConfig
        print("✅ VidGen modules imported successfully (with path fix)")
    except ImportError as e2:
        print(f"⚠️ Path-fixed import also failed: {e2}")
        print("🔄 Using inline implementation for demo...")
        
        # Strategy 3: Inline implementation for demo
        class VideoGenConfig:
            def __init__(self):
                self.gemini_api_key = os.getenv('GEMINI_API_KEY', '')
                self.output_dir = 'colab_output'
                self.fps = 24
                self.resolution = (1280, 720)
                self.duration_per_scene = 4.0
                self.device = 'cuda' if 'torch' in sys.modules and hasattr(sys.modules['torch'], 'cuda') and sys.modules['torch'].cuda.is_available() else 'cpu'
                os.makedirs(self.output_dir, exist_ok=True)
        
        def vidgen_main(script, config):
            print("🎬 VidGen Inline Demo - Processing script...")
            print(f"📝 Script: {script[:100]}...")
            return "inline_demo_completed"

# Create a comprehensive sample script
sample_script = """
Title: "AI in the Future"

Scene 1: Introduction
A futuristic cityscape with towering skyscrapers and flying cars zooming between buildings.
Narrator: "Welcome to the year 2030, where artificial intelligence has transformed our world in ways we never imagined."

Scene 2: AI Assistant
A friendly AI hologram materializes in a modern smart home, with soft blue light illuminating the room.
AI Assistant: "Good morning, Sarah! I've analyzed traffic patterns and optimized your route to work. Your coffee is ready, and I've prepared your daily schedule based on your energy levels."

Scene 3: Workplace Innovation
In a collaborative workspace, humans and AI work side by side on holographic displays.
Narrator: "Collaboration between humans and AI has reached new heights, enhancing creativity and solving complex problems."

Scene 4: Conclusion
The camera pans across a harmonious blend of advanced technology and preserved natural landscapes.
Narrator: "The future is here, and it's more amazing and sustainable than we ever imagined. Welcome to the age of responsible AI."
"""

print("✅ Sample script created")
print("=" * 60)
print(sample_script)
print("=" * 60)
print(f"📊 Script length: {len(sample_script)} characters")
print(f"🎬 Estimated scenes: {len([line for line in sample_script.split('\n') if line.strip().startswith('Scene')])}")

In [ ]:
# Enhanced VidGen configuration and execution
print("🚀 Starting enhanced video generation...")

# Configure VidGen with enhanced settings
config = VideoGenConfig()
config.gemini_api_key = GEMINI_API_KEY
config.duration_per_scene = 4.0  # 4 seconds per scene
config.output_dir = 'colab_output'
config.fps = 24

print(f"🎮 Device: {config.device}")
print(f"📁 Output directory: {config.output_dir}")
print(f"🔑 API key status: {'Configured' if config.gemini_api_key and config.gemini_api_key != 'demo_mode' else 'Demo Mode'}")

# Enhanced execution with progress tracking
try:
    print("\n🎬 Starting video generation pipeline...")
    
    # Step 1: Process script
    print("📝 Step 1: Processing script...")
    processed_script = sample_script
    
    # Step 2: Generate video
    print("🎥 Step 2: Generating video content...")
    output_path = vidgen_main(processed_script, config)
    
    print(f"\n✅ Video generation complete!")
    print(f"📹 Output file: {output_path}")
    
    # Step 3: Display results
    if output_path and output_path != "demo_completed" and output_path != "inline_demo_completed":
        if os.path.exists(output_path) and output_path.endswith('.mp4'):
            print("🎬 Displaying generated video:")
            
            # Display video in Colab
            try:
                from IPython.display import Video, display
                display(Video(output_path, width=640, height=360, embed=True))
                
                # Show file info
                file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
                print(f"📊 File size: {file_size:.2f} MB")
                
            except Exception as e:
                print(f"⚠️ Could not display video: {e}")
                print(f"📁 Video saved at: {output_path}")
        else:
            print(f"⚠️ Video file not found: {output_path}")
    else:
        print("🎭 Demo completed successfully!")
        print("📝 This was a demonstration of the VidGen workflow.")
        print("🔗 For full video generation, ensure you have:")
        print("   • A valid Gemini API key")
        print("   • Sufficient GPU memory")
        print("   • All dependencies properly installed")
        
        # Show what would happen in full mode
        print("\n🔮 In full mode, VidGen would:")
        print("   1. 🤖 Analyze script with Gemini AI")
        print("   2. 🎨 Generate character images with Stable Diffusion")
        print("   3. 🔊 Create voice narration with Bark TTS")
        print("   4. 🎵 Generate background music and effects")
        print("   5. 🎬 Assemble everything into a final video")
        
except Exception as e:
    print(f"❌ Error during video generation: {e}")
    print("\n🔧 Troubleshooting tips:")
    print("   • Restart runtime if you're getting memory errors")
    print("   • Check that GPU is enabled in Runtime settings")
    print("   • Verify your Gemini API key is correct")
    print("   • Try running the setup cells again")
    
    # Show system status for debugging
    print("\n📊 System status:")
    try:
        import torch
        print(f"   • PyTorch CUDA: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"   • GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
    except:
        print("   • PyTorch: Not available")
    
    import psutil
    memory = psutil.virtual_memory()
    print(f"   • RAM Usage: {memory.percent:.1f}%")
    print(f"   • Available RAM: {memory.available / 1e9:.1f}GB")

print("\n🎆 Video generation process completed!")

## 5. 🎨 Advanced Video Generation

Explore advanced features and customization options (requires full VidGen setup).

In [ ]:
from vidgen.services.script_parser import ScriptParser
from vidgen.services.gemini_api import call_gemini_api
from vidgen.models.video import generate_video_segment
from vidgen.models.audio import generate_background_audio
from vidgen.services.video_assembler import assemble_video
import json

# Advanced script with multiple characters
advanced_script = """
Title: "The Digital Revolution"

A bustling tech startup office with developers coding.
[Background: Upbeat tech music]

ALEX: "We're on the verge of something revolutionary."

Close-up of code streaming across multiple monitors.

SARAH: "This AI will change how we interact with technology forever."

A sleek AI interface with flowing data visualizations.

NARRATOR: "Innovation never sleeps in the digital age."
"""

# Process script with Gemini
print("🤖 Processing script with Gemini AI...")
gemini_response = call_gemini_api(advanced_script, GEMINI_API_KEY, segment_duration=6)

if gemini_response:
    # Parse the structured response
    parser = ScriptParser()
    parsed_script = parser.parse_gemini_response(gemini_response)
    
    print(f"📊 Parsed {len(parsed_script)} video segments")
    
    # Generate video segments
    video_segments = []
    face_cache = {}  # Cache character faces
    
    for i, segment in enumerate(parsed_script):
        print(f"🎬 Generating segment {i+1}/{len(parsed_script)}...")
        
        segment_path = generate_video_segment(
            segment, 
            face_cache=face_cache,
            default_duration=6
        )
        
        if segment_path:
            video_segments.append(segment_path)
            print(f"✅ Segment {i+1} completed")
    
    # Generate background audio
    print("🎵 Generating background audio...")
    bg_audio = generate_background_audio("Upbeat tech music")
    
    # Assemble final video
    print("🎞️ Assembling final video...")
    final_video = assemble_video(
        video_segments=video_segments,
        background_audio=bg_audio,
        output_filename="advanced_video.mp4"
    )
    
    print(f"🎉 Advanced video completed: {final_video}")
else:
    print("❌ Failed to process script with Gemini")

## 5. 🖥️ Launch Web Interface

Use the Gradio web interface for interactive video generation.

In [ ]:
from vidgen.ui.gradio_app import create_gradio_interface

# Create and launch Gradio interface
interface = create_gradio_interface()

# Launch with public link for Colab
interface.launch(
    share=True,  # Creates public link
    debug=True,  # Enable debugging
    server_name="0.0.0.0",  # Allow external connections
    server_port=7860  # Default Gradio port
)

## 6. 📁 Download Generated Videos

Download your generated videos to your local machine.

In [ ]:
from google.colab import files
import os
import glob

# List all generated videos
video_dir = "/content/vidgen/outputs/videos"
video_files = glob.glob(os.path.join(video_dir, "*.mp4"))

print(f"📹 Found {len(video_files)} generated videos:")
for video in video_files:
    print(f"  - {os.path.basename(video)}")

# Download all videos
if video_files:
    print("\n⬇️ Downloading videos...")
    for video in video_files:
        try:
            files.download(video)
            print(f"✅ Downloaded: {os.path.basename(video)}")
        except Exception as e:
            print(f"❌ Failed to download {video}: {e}")
else:
    print("No videos found. Generate some videos first!")

## 7. 🧪 Test Individual Components

Test individual VidGen components for debugging and experimentation.

In [ ]:
# Test image generation
print("🖼️ Testing image generation...")
from vidgen.models.image import generate_character_image

image_path = generate_character_image(
    "A friendly scientist in a white lab coat, professional headshot",
    seed=42
)
print(f"Image generated: {image_path}")

# Test TTS generation
print("\n🗣️ Testing text-to-speech...")
from vidgen.models.tts import generate_tts_audio

tts_path = generate_tts_audio("Hello, this is a test of the text to speech system.")
print(f"TTS audio generated: {tts_path}")

# Test audio generation
print("\n🎵 Testing background audio generation...")
from vidgen.models.audio import generate_background_audio

audio_path = generate_background_audio("Ambient forest sounds with gentle bird calls")
print(f"Background audio generated: {audio_path}")

print("\n✅ All components tested successfully!")

## 8. 📊 Performance Monitoring

Monitor GPU usage and performance metrics.

In [ ]:
import torch
import psutil
import time

def monitor_resources():
    """Monitor system resources during video generation."""
    
    # GPU information
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        gpu_memory_used = torch.cuda.memory_allocated(0) / 1e9
        gpu_memory_cached = torch.cuda.memory_reserved(0) / 1e9
        
        print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
        print(f"📊 GPU Memory: {gpu_memory_used:.1f}GB / {gpu_memory:.1f}GB")
        print(f"💾 GPU Cached: {gpu_memory_cached:.1f}GB")
    else:
        print("❌ No GPU available")
    
    # CPU and RAM information
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()
    
    print(f"💻 CPU Usage: {cpu_percent:.1f}%")
    print(f"🧠 RAM Usage: {memory.percent:.1f}% ({memory.used/1e9:.1f}GB / {memory.total/1e9:.1f}GB)")
    
    # Disk space
    disk = psutil.disk_usage('/content')
    print(f"💾 Disk Usage: {disk.percent:.1f}% ({disk.used/1e9:.1f}GB / {disk.total/1e9:.1f}GB)")

print("📊 Current System Status:")
monitor_resources()

# Clear GPU cache if needed
if torch.cuda.is_available():
    print("\n🧹 Clearing GPU cache...")
    torch.cuda.empty_cache()
    print("✅ GPU cache cleared")

## 🎯 Tips for Best Results in Colab

### 🚀 Performance Tips
1. **Use GPU Runtime**: Always enable GPU for faster generation
2. **Monitor Memory**: Check GPU memory usage to avoid OOM errors
3. **Cache Models**: Models are cached automatically to speed up subsequent runs
4. **Batch Processing**: Generate multiple segments efficiently

### 📝 Script Writing Tips
1. **Be Descriptive**: Detailed scene descriptions produce better images
2. **Character Consistency**: Use consistent character descriptions
3. **Audio Cues**: Include background music and sound effect descriptions
4. **Pacing**: Aim for 5-10 second segments for best results

### 🔧 Troubleshooting
- **Out of Memory**: Restart runtime and clear GPU cache
- **Slow Generation**: Check GPU is enabled and models are cached
- **API Errors**: Verify Gemini API key is correctly set
- **Audio Issues**: Ensure FFmpeg is properly installed

---

**🎉 Happy Video Creating with VidGen! 🎬**